In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import os
import shutil
import math
import cv2
from tqdm import tqdm

In [2]:
IMAGE_DIR = 'E:/Datasets/BienSoXeCongHai/images'

## Tran/Test split

In [3]:
TRAIN_DIR = 'E:/Datasets/BienSoXeCongHai/train_images'
VAL_DIR = 'E:/Datasets/BienSoXeCongHai/val_images'
NUM_VAL_SAMPLES = 32

In [7]:

images = os.listdir(IMAGE_DIR)
images = np.array(images)
val_idxs = np.random.choice(len(images), 32, replace=False)
val_images = images[val_idxs]
train_images = [image for image in images if image not in val_images]

for image in val_images:
    shutil.copy(IMAGE_DIR + '/' + image, VAL_DIR + '/' + image)
for image in train_images:
    shutil.copy(IMAGE_DIR + '/' + image, TRAIN_DIR + '/' + image)

## Image augmentation

In [4]:
augmentor = iaa.Sequential([
    iaa.OneOf([
        iaa.Affine(scale=(0.8, 1.2), translate_percent={'x': (-0.2, 0.2), 'y': (-0.2, 0.2)}),
        iaa.PerspectiveTransform(scale=0.12),
        iaa.PiecewiseAffine(scale=0.02)
    ]),
    iaa.Sometimes(0.5, [
        iaa.ElasticTransformation(sigma=5.0, alpha=20.0)
    ]),
    iaa.OneOf([
        iaa.CoarseDropout(p=0.075, size_percent=0.1),
        iaa.CoarseDropout(p=0.075, size_percent=0.1, per_channel=True),
        iaa.FastSnowyLandscape(lightness_multiplier=1.5),
        iaa.Clouds(),
        iaa.Fog(),
        iaa.MotionBlur(k=15),
    ])
])

In [ ]:
for image in train_images[:10]:
    image = plt.imread(TRAIN_DIR + '/' + train_images[-7])
    image = augmentor.augment_image(image)
    plt.imshow(image)
    plt.show()

In [5]:
NUM_EXAMPLES = 1000

In [6]:
AUGMENTED_DIR = 'E:/Datasets/BienSoXeCongHai/augmented_images'
train_images = os.listdir(TRAIN_DIR)
train_images = [image for image in train_images if image.endswith('.JPG')]

for i in tqdm(range(math.ceil(NUM_EXAMPLES / len(train_images)))):
    for file in tqdm(train_images, leave=False):
        image = plt.imread(TRAIN_DIR + '/' + file)
        image = augmentor.augment_image(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(AUGMENTED_DIR + '/' + file.replace('.', '.{}.'.format(i)), image)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [04:18<00:00, 32.57s/it]


## Generate license plates

In [8]:
NUM_OF_SAMPLE = 50000
SAVE_DIR_1 = 'E:/Datasets/Vietnamese Motorbike License Plate/old_license_plate'

In [7]:

for i in tqdm(range(NUM_OF_SAMPLE)):
    license = random_license_number()
    image = create_plate(license)
    image.save('{}/{}.jpg'.format(SAVE_DIR_1, license.replace('_', '').replace('\n', '')))

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 154.00it/s]


## Generate license plates in scene

In [9]:
license_plates = os.listdir(SAVE_DIR_1)
license_plates = [f for f in license_plates if f.endswith('.jpg')]
SAVE_DIR_2 = 'E:/Datasets/Vietnamese Motorbike License Plate/images'

generated = os.listdir(SAVE_DIR_2)
license_plates = [f for f in license_plates if f not in generated]

for plate in tqdm(license_plates):
    image = Image.open('{}/{}'.format(SAVE_DIR_1, plate))
    image = add_random_bg(image)
    while True:
        try:
            image = add_noise(image)
            break
        except:
            pass
    image = Image.fromarray(image)
    image.save('{}/{}'.format(SAVE_DIR_2, plate))

100%|██████████████████████████████████████████████████████████████████████████| 49206/49206 [6:27:50<00:00,  1.66it/s]


## Generate directly license plates in scene

In [ ]:
NUM_OF_SAMPLE = 50000

licenses = []
for _ in range(NUM_OF_SAMPLE):
    license = random_license_number()
    licenses.append(license)
    
SAVE_DIR_2 = 'E:/Datasets/Vietnamese Motorbike License Plate/images2'

for license in tqdm(licenses):
    image = create_plate(license)
    image = add_random_bg(image)
    while True:
        try:
            image = add_noise(image)
            break
        except:
            pass
    image = Image.fromarray(image)
    image.save('{}/{}'.format(SAVE_DIR_2, license.replace('_', '').replace('\n', '')))

In [6]:
SAVE_DIR_2 = 'E:/Datasets/Vietnamese Motorbike License Plate/images'

In [10]:
from tqdm import tqdm
import os

In [16]:
for file in tqdm(os.listdir(SAVE_DIR_1)):
    a = file.split('.jpg')[0]
    a = a.replace('-', '').replace('.', '')
    a = a + '.jpg'
    os.rename(SAVE_DIR_1 + '/' + file, SAVE_DIR_1 + '/' + a)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 860.10it/s]
